## 第七章、特征选择

这一章的目的是根据现有样本的特征，在其中提取出对样本分类有显著影响的特征。

前几节介绍了判断某个特征是否对分类有显著影响的数学方法。这里不再赘述了。

下面是书中介绍的特征选择的算法。

### 一.特征选择的最优算法

这个算法是基于深度优先搜索的选择特征算法，通过选取最大化准则函数值进行剪枝，这样可以大大减少搜索次数。但是算法题刷多了就会明白，就算剪枝剪得再多，搜索的复杂度还是指数级别的，还是非常的慢。书上也说了这个算法很慢，通常也没有人会用。

### 二、特征选择的次优算法

书上介绍了四种算法：单独最优特征的组合、顺序前进法、顺序后退法、增l减r法（l-r法）。四种算法思想都很简单，

### 三、特征选择的遗传算法

遗传算法是通过模拟生物染色体遗传进化来实现特征选择，在特征维数很高的时候效果很好。这个算法应用非常广，但是书上也只是简单介绍了一下。

### 四、以分类性能为准则的特征选择方法

这个方法的思路就是用分类器进行特征选择，书上介绍了R-SVM和SVM-RFE方法。

这两个方法是利用了支持向量机在样本维数很高并且样本较少的情况下表现很好的特性，通过SVM训练之后的权值数据来判断特征对分类贡献大小。

具体的方法如下：

1. 用**线性**支持向量机对样本数据进行训练，得到对应对偶问题的$\boldsymbol{\alpha}$
2. 通过$\boldsymbol{\alpha}$算出原始问题的系数$\mathbf{w}$

$$
\mathbf{w} = \sum_{i=1}^N\alpha_iy_i\mathbf{x}_i
$$

说明一下，sklearn中的线性支持向量机LinearSVC中有直接算出$\mathbf{w}$的API，不用先计算出$\boldsymbol{\alpha}$，下面的代码中我会直接用这个API。

3. 计算每个特征对分类的贡献。对R-SVM和SVM-RFE来说，分类的贡献公式分别是：

$$\begin{align}
s_j^{\text{R-SVM}} &= w_j(m_j^+ - m_j^-) \\
s_j^{\text{SVM-RFE}} &= w_j^2 \quad j = 1,\cdots,d
\end{align}$$

4. 接下来就根据不同特征的贡献选择特征

实现代码如下：

首先导包并导入数据（数据用之前用过的手写数字识别）

In [1]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

X = np.load("./data/digit_0_1_X.npy")
y = np.load("./data/digit_0_1_y.npy")
y = y.reshape(y.shape[0])

X_train, X_test, y_train, y_test = train_test_split(X[:1000], y[:1000], test_size=0.2)

print("训练集大小如下：")
print(X_train.shape)
print(y_train.shape)
print("测试集大小如下：")
print(X_test.shape)
print(y_test.shape)

训练集大小如下：
(800, 400)
(800,)
测试集大小如下：
(200, 400)
(200,)


接下来用线性支持向量机训练上述数据

In [2]:
linear_SVC = LinearSVC()

linear_SVC.fit(X_train, y_train)

w = linear_SVC.coef_.reshape(X_train[0].shape[0])

print("训练后的w大小为：", w.shape)

print("在测试集上正确率为：", linear_SVC.score(X_test, y_test))

训练后的w大小为： (400,)
在测试集上正确率为： 0.995


接下来计算不同特征的贡献。书上说R-SVM更好，所以下面计算R-SVM的贡献。

In [3]:
m_1 = np.lib.average(X_train[:50], 0)
m_2 = np.lib.average(X_train[50:], 0)

s = w * (m_1 - m_2)

print("s的大小为：", s.shape)
print("s的最大值为：", np.max(s), "\ns的最小值为：", np.min(s))

s的大小为： (400,)
s的最大值为： 0.00724647354728087 
s的最小值为： -0.01821647548504886


下面删掉贡献值最小的200个特征特征（总共有400个特征），再重新训练测试正确率

In [4]:
# 统计贡献值最小的两百个特征
# 下面的这段代码可能不太容易看懂，不懂也没关系，只需要知道作用就行了
s_temp = np.concatenate((s.reshape(s.shape[0], 1), np.arange(0, s.shape[0]).reshape(s.shape[0], 1)), 1)
s_temp.sort(0)
index_to_delete = []
for row in s_temp:
    index_to_delete.append(int(row[1]))

# 删掉贡献值最小的两百个特征
X_train = np.delete(X_train, index_to_delete[:200], axis=1)
X_test = np.delete(X_test, index_to_delete[:200], axis=1)

print("删除后的训练集大小：", X_train.shape)
print("删除后的测试集大小：", X_test.shape)

linear_SVC.fit(X_train, y_train)

print("删除两百个特征后的正确率：", linear_SVC.score(X_test, y_test))

删除后的训练集大小： (800, 200)
删除后的测试集大小： (200, 200)
删除两百个特征后的正确率： 0.995


很明显删除了一半的特征之后正确率还是没有变化，这说明我们使用的特征选择方法是很有效的。